In [1]:
# validation script
path = r"/home/wslvivek/Desktop/level2/pysh_v2/JPL_input/"                      # Path to JPL GRACE data
path_tn14 = r"/home/wslvivek/Desktop/level2/pysh_v2/JPL_TN_files/TN-14_C30_C20_GSFC_SLR.txt"    # Path to TN14
path_tn13 = r"/home/wslvivek/Desktop/level2/pysh_v2/JPL_TN_files/TN-13_GEOC_JPL_RL06.txt"       # Path to TN13
source='jpl'

In [2]:
import os
path_master = os.getcwd()                                            # current directory
path_functions = os.path.join(path_master,"pysh_funcs")        # directory of the sh bundle functions
path_output = os.path.join(os.path.dirname(path_master),"output")    # directory for output
from reader_replacer_jpl import reader_replacer_jpl
from clm2sc import clm2sc
from scrpit import tws_cal
from basin_avg import basin_avg
import numpy as np
import pandas as pd
import xarray as xr
import warnings

In [3]:
sh_clm, dates_start, dates_end, no_of_files = reader_replacer_jpl(path, path_tn14, path_tn13)

JPL
Reading into clm format complete!
Number of files read: 201
Starting replacement
found: 37
2002-04-04 2002-04-04 0
2002-05-02 2002-05-02 1
2002-08-01 2002-08-01 2
2002-09-01 2002-09-01 3
2002-10-01 2002-10-01 4
2002-11-01 2002-11-01 5
2002-12-01 2002-12-01 6
2003-01-01 2003-01-01 7
2003-02-01 2003-02-01 8
2003-03-01 2003-03-01 9
2003-04-01 2003-04-01 10
2003-05-01 2003-05-01 11
2003-07-01 2003-07-01 12
2003-08-01 2003-08-01 13
2003-09-01 2003-09-01 14
2003-10-01 2003-10-01 15
2003-11-01 2003-11-01 16
2003-12-01 2003-12-01 17
2004-01-01 2004-01-01 18
2004-02-04 2004-02-04 19
2004-03-01 2004-03-01 20
2004-04-01 2004-04-01 21
2004-05-01 2004-05-01 22
2004-06-01 2004-06-01 23
2004-11-01 2004-11-01 28
2004-12-01 2004-12-01 29
2005-01-01 2005-01-01 30
2005-02-01 2005-02-01 31
2005-03-01 2005-03-01 32
2005-04-01 2005-04-01 33
2005-05-01 2005-05-01 34
2005-06-01 2005-06-01 35
2005-07-01 2005-07-01 36
2005-08-01 2005-08-01 37
2005-09-01 2005-09-01 38
2005-10-01 2005-10-01 39
2005-11-01 2005

In [4]:
raw_sc = clm2sc(sh_clm);

Conversion into clm format complete


In [5]:
raw_sc = clm2sc(sh_clm);
import scipy.io
scipy.io.savemat('/home/wslvivek/Desktop/level2/pysh_v2/validation/input for matlab/raw_sc.mat', {"raw_sc": raw_sc})

Conversion into clm format complete


In [ ]:
lmax = 96
gs = 1
half_rad_gf = 500
lon = np.arange(-180,180,gs)
lat = np.arange(89,-91,-gs)
# dates = pd.to_datetime(pd.Series(dates_start), format='%Y-%m-%d').dt.strftime('%d-%m-%Y') \
#         + "    " \
#         +pd.to_datetime(pd.Series(dates_end), format='%Y-%m-%d').dt.strftime('%d-%m-%Y')
dates = pd.to_datetime(pd.Series(dates_start), format='%Y-%m-%d')#.dt.strftime('%d-%m-%Y')
print('dates read! \n')
dates.to_csv(path_output+'/dates.csv')
SH_long_mean_jpl = np.load(path_master+'/long_mean/SH_long_mean_jpl.npy')    # load the long term mean SH coeffs---> JPL 
delta_sc = raw_sc - SH_long_mean_jpl

In [ ]:
%%time
tws_fields = tws_cal(delta_sc,lmax,gs,half_rad_gf,no_of_files,path_functions)
tws_fields = np.float32(tws_fields)

In [ ]:
ds = xr.Dataset(
    data_vars=dict(
        tws=(["time","lat", "lon"], tws_fields)
    ),
    coords = {
        "time":(('time'),dates),
        "lat":lat,
        "lon":lon },
    attrs=dict(description="TWS Anomaly corresponding to long term (2004-2010) mean \n lmax=96 and half radius of Gaussian filter = 500Km"),
)
ds

In [ ]:
ds.tws.isel(time=200).plot()